In [1]:
import pandas as pd

df = pd.read_csv('tcga_survival.csv')
df

,patient_id,stage_m,stage_t,stage_n,race,sex,subtype,years_at_dx,is_adjuvant_rt_given,menopausal_status,years_to_survival_followup,event
0,TCGA-2F-A9KO,M0,T3,N1,White,Male,BLCA,63.854894,Yes,NaN,2.010133,death
1,TCGA-2F-A9KP,MX,T3,N2,White,Male,BLCA,66.880219,No,NaN,0.996851,death
2,TCGA-2F-A9KQ,M0,T3,N0,White,Male,BLCA,69.155373,No,NaN,7.903601,NaN
3,TCGA-2F-A9KR,M0,T3,N0,Missing,Female,BLCA,59.816564,Yes,NaN,8.716966,death
4,TCGA-2F-A9KT,M0,T2,N0,White,Male,BLCA,83.559206,No,NaN,9.075722,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
4076,TCGA-61-2614,NaN,NaN,NaN,White,Female,OV,71.572895,No,NaN,0.717513,death
4077,TCGA-OY-A56P,NaN,NaN,NaN,Black,Female,OV,48.131417,No,NaN,3.305491,recurrence
4078,TCGA-OY-A56Q,NaN,NaN,NaN,Black,Female,OV,78.365503,Yes,NaN,1.577434,NaN
4079,TCGA-VG-A8LO,NaN,NaN,NaN,Black,Female,OV,55.507187,Yes,NaN,0.065726,death


In [2]:
df.describe()

,years_at_dx,years_to_survival_followup
count,4000.000000,4058.000000
mean,63.368468,2.883659
std,11.769263,2.690207
min,26.573580,0.000000
25%,55.473648,1.109818
50%,63.728953,2.077229
75%,71.967146,3.797754
max,89.998631,23.565658


In [3]:
df.isnull().sum()  # Count missing values per column

patient_id                       0
stage_m                       1105
stage_t                        626
stage_n                        667
race                             0
sex                              0
subtype                          0
years_at_dx                     81
is_adjuvant_rt_given             0
menopausal_status             3108
years_to_survival_followup      23
event                         2654
dtype: int64

In [4]:
for col in df.select_dtypes(include=["object"]).columns:
    print(f"{col}: {df[col].nunique()} unique values")
    print(df[col].value_counts(), "\n")

patient_id: 4081 unique values
patient_id
TCGA-2F-A9KO    1
TCGA-56-7822    1
TCGA-56-1622    1
TCGA-56-5897    1
TCGA-56-5898    1
               ..
TCGA-EW-A1IZ    1
TCGA-EW-A1J1    1
TCGA-EW-A1J2    1
TCGA-EW-A1J3    1
TCGA-WR-A838    1
Name: count, Length: 4081, dtype: int64 

stage_m: 3 unique values
stage_m
M0    2212
MX     634
M1     130
Name: count, dtype: int64 

stage_t: 8 unique values
stage_t
T2          1597
T3          1060
T1mi_a_b     358
T1c          223
T4           208
TX             7
T0             1
Tis            1
Name: count, dtype: int64 

stage_n: 5 unique values
stage_n
N0    2028
N1     827
N2     394
N3      92
NX      73
Name: count, dtype: int64 

race: 8 unique values
race
White                                        2687
Missing                                       747
Black                                         390
Asian                                         155
[not available]                                92
American indian or alaska native  

Modeling

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from lifelines.utils import datetimes_to_durations
import numpy as np

# Encode categorical variables
categorical_columns = ['race', 'sex', 'subtype', 'is_adjuvant_rt_given', 'stage_m', 'stage_t', 'stage_n']
le = LabelEncoder()

# TODO Other ways of encoding columns (especially stages and subtype that have functional meaning as well as adjuvant RT)
for col in categorical_columns:
    df[col] = le.fit_transform(df[col].astype(str))

# Convert the target variable 'event' to binary (death=1, others=0)
df['event_binary'] = df['event'].apply(lambda x: 1 if x == "death" else 0)

# Split into features (X) and target (y)
# TODO How to include menopause status back in
X = df[['stage_m', 'stage_t', 'stage_n', 'race', 'sex', 'subtype', 'years_at_dx', 
        'is_adjuvant_rt_given']]
y = df[['event_binary', 'years_to_survival_followup']]

In [6]:
X

,stage_m,stage_t,stage_n,race,sex,subtype,years_at_dx,is_adjuvant_rt_given
0,0,4,1,5,1,0,63.854894,1
1,2,4,2,5,1,0,66.880219,0
2,0,4,0,5,1,0,69.155373,0
3,0,4,0,3,0,0,59.816564,1
4,0,3,0,5,1,0,83.559206,0
...,...,...,...,...,...,...,...,...
4076,3,8,5,5,0,5,71.572895,0
4077,3,8,5,2,0,5,48.131417,0
4078,3,8,5,2,0,5,78.365503,1
4079,3,8,5,2,0,5,55.507187,1


In [7]:
y.sum()

event_binary                   1200.000000
years_to_survival_followup    11701.886896
dtype: float64

In [8]:
from lifelines import CoxPHFitter

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

cph = CoxPHFitter()
X_train_merged = X_train.copy()
X_train_merged['duration'] = y_train['years_to_survival_followup']
X_train_merged['event'] = y_train['event_binary']

print(X_train_merged.isna().sum())
X_train_merged = X_train_merged.apply(pd.to_numeric, errors='coerce')
X_train_merged = X_train_merged.dropna()

cph.fit(X_train_merged, duration_col='duration', event_col='event')
cph.print_summary()

stage_m                  0
stage_t                  0
stage_n                  0
race                     0
sex                      0
subtype                  0
years_at_dx             60
is_adjuvant_rt_given     0
duration                16
event                    0
dtype: int64


<lifelines.CoxPHFitter: fitted with 2789 total observations, 1977 right-censored observations>
             duration col = 'duration'
                event col = 'event'
      baseline estimation = breslow
   number of observations = 2789
number of events observed = 812
   partial log-likelihood = -5533.56
         time fit was run = 2025-03-31 04:07:12 UTC

---
                      coef exp(coef)  se(coef)  coef lower 95%  coef upper 95% exp(coef) lower 95% exp(coef) upper 95%
covariate                                                                                                             
stage_m              -0.19      0.83      0.05           -0.28           -0.10                0.75                0.91
stage_t               0.18      1.20      0.03            0.12            0.24                1.13                1.27
stage_n               0.19      1.21      0.03            0.13            0.26                1.14                1.29
race                 -0.01      0.99      0.03           -0.07            0.06                0.93                1.06
sex                   0.42      1.52      0.08            0.26            0.59                1.29                1.80
subtype              -0.07      0.94      0.03           -0.12           -0.02                0.89                0.98
years_at_dx           0.03      1.03      0.00            0.02            0.04                1.02                1.04
is_adjuvant_rt_given  0.01      1.01      0.07           -0.13            0.15                0.88                1.16

                      cmp to     z      p  -log2(p)
covariate                                          
stage_m                 0.00 -3.98 <0.005     13.81
stage_t                 0.00  6.13 <0.005     30.06
stage_n                 0.00  6.09 <0.005     29.70
race                    0.00 -0.21   0.84      0.26
sex                     0.00  4.99 <0.005     20.66
subtype                 0.00 -2.55   0.01      6.56
years_at_dx             0.00  9.38 <0.005     67.05
is_adjuvant_rt_given    0.00  0.18   0.86      0.22
---
Concordance = 0.68
Partial AIC = 11083.11
log-likelihood ratio test = 302.07 on 8 df
-log2(p) of ll-ratio test = 198.74

In [9]:
from lifelines.utils import concordance_index

# Merge test data with features
X_test_merged = X_test.copy()
X_test_merged['duration'] = y_test['years_to_survival_followup']
X_test_merged['event'] = y_test['event_binary']
X_test_merged = X_test_merged.apply(pd.to_numeric, errors='coerce')

# Predict the risk scores on the test set
y_pred = cph.predict_partial_hazard(X_test_merged)
X_test_merged['pred'] = y_pred
X_test_merged = X_test_merged.dropna()

# Calculate C-index
c_index = concordance_index(X_test_merged['duration'], X_test_merged['pred'], X_test_merged['event'])
print(f"C-index: {c_index}")

C-index: 0.29415600835806593
